# Fundamentals of Analog-to-Digital Conversion (ADC)

Analog-to-Digital Conversion (ADC) is the process of converting a **continuous analog** signal into a digital form that can be processed by computers and digital systems. Key concepts include **quantization**, **oversampling**, **aliasing**, and **signal-to-noise ratios**.

## 1. **Quantization**

Quantization is the process of mapping a continuous analog signal to a finite set of discrete levels. The number of quantization levels is determined by the ADC's bit depth. For an ADC with **N bits**, the total number of quantization levels, $L$, is:

$$
L = 2^N
$$

### Step Size (LSB)
The range of the input signal is typically limited to a maximum value of $\pm V_{max}$, resulting in a total full-scale range (FSR):

$$
FSR = 2 \cdot V_{max}
$$

The quantization step size (also called Least Significant Bit, LSB) is the difference between two adjacent quantization levels and can be calculated as:

$$
\Delta = \frac{FSR}{L} = \frac{2V_{max}}{2^N} = \frac{V_{max}}{2^{N-1}}
$$

This step size is the smallest change the ADC can detect. 

### Quantization Process

To quantize a continuous signal, we first normalize it to the ADC's range. Given an input signal $x(t)$ (which can vary continuously between $-V_{max}$ and $+V_{max}$), we map it to a set of discrete levels. The quantization process involves:

1. **Normalization and Index Calculation**: We map the continuous signal $x(t)$ into the range of integers corresponding to the quantization levels. Then, the quantization index for a value of the signal $x(t)$ is calculated by:

$$
i = \left\lfloor \frac{x(t) - (-V_{max})}{\Delta} \right\rfloor = \left\lfloor \frac{x(t) + V_{max}}{\Delta} \right\rfloor
$$

Where $i$ is the index of the quantization level and $\Delta$ is the step size (LSB). The floor function $\lfloor \cdot \rfloor$ ensures that the signal is mapped to the closest quantization level below the input value.

2. **Clipping**: To ensure that the quantization index stays within the valid range of levels, we clip the index to the range $[0, L-1]$:

$$
i_{\text{clipped}} = \max(0, \min(i, L-1))
$$

3. **Reconstruction**: Once the quantization index $i_{\text{clipped}}$ is calculated, the quantized value is reconstructed by mapping the index back to the corresponding quantized voltage level:

$$
x_{\text{quantized}}(t) = V_{min} + (i_{\text{clipped}} + 0.5) \Delta
$$

Here, $V_{min} = -V_{max}$, and adding $0.5 \Delta$ centers the quantized value within each quantization interval.

### Quantization Error and **Signal-to-Quantization-Noise Ratio (SQNR)**

The difference between the original signal and its quantized value is called the **quantization error**. This error is bounded by half the step size:

$$
-\frac{\Delta}{2} \le e_q \le \frac{\Delta}{2}
$$

Quantization error introduces noise in the signal, but it is typically small, especially at high bit depths. The Signal-to-Quantization-Noise Ratio (SQNR) measures the ratio of signal power to quantization noise. For an ideal N-bit ADC and a full-scale **sine wave input**, the SQNR can be expressed as:

$$
SQNR_{dB} = 6.02N + 1.76
$$

Where:
- $N$ is the number of bits in the ADC.

This formula shows that **each additional bit increases the SQNR by approximately 6 dB**.



### 2. **Bandwidth (BW) Process Gain**

When oversampling a signal (sampling at a rate greater than twice the signal bandwidth) and digitally filtering it to a narrower bandwidth, noise outside the band of interest is removed. This improves SQNR.

The bandwidth process gain is given by:

$$
PG_{BW} = 10 \cdot \log_{10}\left( \frac{f_s / 2}{BW_{target}} \right)
$$

Where:
- $f_s / 2$ is the Nyquist bandwidth.
- $BW_{target}$ is the bandwidth of interest.

**Effect:** Doubling the sampling rate for a fixed signal bandwidth increases SQNR by 3 dB.


### 3. **FFT Process Gain**

When analyzing a signal using an FFT, quantization noise is distributed across all FFT bins, while the signal energy remains concentrated in one or a few bins.

The FFT process gain is:

$$
PG_{FFT} = 10 \cdot \log_{10}\left( \frac{N_{FFT}}{2} \right)
$$

Where, $N_{FFT}$ is the number of FFT points. This does not remove noise, but it lowers the apparent noise floor per bin by narrowing the resolution bandwidth.


### 4. **Aliasing**

Aliasing occurs when a signal is undersampled, meaning the sampling rate is too low to accurately capture the signal’s frequency content. High-frequency components fold back into lower frequencies, distorting the signal.

To avoid aliasing, the sampling rate must satisfy:

$$
f_s \ge 2 f_{max}
$$

This is known as the Nyquist criterion.


### 5. **Spurious-Free Dynamic Range (SFDR)**

The **Spurious-Free Dynamic Range (SFDR)** is an important performance metric for ADCs, measuring the difference between the fundamental signal (usually the desired carrier or input signal) and the highest spurious signal, often caused by harmonic distortion or non-idealities in the ADC.

SFDR is defined as:

$$
SFDR = 20 \cdot \log_{10}\left( \frac{A_{signal}}{A_{spurious}} \right)
$$

Where:
- $A_{signal}$ is the amplitude of the fundamental signal.
- $A_{spurious}$ is the amplitude of the largest spurious tone or harmonic (distortion component).

#### SFDR with respect to Full Scale

In many cases, SFDR is specified relative to the **full-scale signal** (FSR), which is the maximum possible input signal that the ADC can handle. When SFDR is referenced to full scale, the formula becomes:

$$
SFDR_{FS} = 20 \cdot \log_{10}\left( \frac{V_{FS}}{A_{spurious}} \right)
$$

Where $V_{FS}$ is the full-scale voltage (i.e., $2 \cdot V_{max}$ for an ADC with a ±$V_{max}$ input range). This indicates how much stronger the signal is compared to the highest spurious signal, expressed in dB.

#### SFDR with respect to the Carrier

Alternatively, SFDR can be defined with respect to the **carrier** signal (the desired signal at the frequency of interest), especially when dealing with modulated signals in communication systems. In this case, SFDR is the ratio between the carrier amplitude and the largest spurious signal at the carrier's frequency.

The key takeaway is that **higher SFDR values indicate better ADC performance**, as it signifies that the desired signal is much stronger than the distortion and noise present in the system.

---


## 1. Quantization, SNQR, and Processing Gain 

In [16]:
# Importing after ensuring packages are installed
import numpy as np
import matplotlib.pyplot as plt
import mplcursors  # Import for click interactivity
from ipywidgets import widgets, interactive_output, VBox, HBox
from IPython.display import display, clear_output
from scipy.fft import fft, fftfreq, fftshift

# Ensure you have run: !pip install ipympl mplcursors
%matplotlib widget

# --- 0. Memory Management ---
plt.close('all')  # <--- This kills the old figure and resets memory

f_signal = 997
oversampling_factor = 100

# --- 2. Plotting Function ---
def plot_adc(bits, signal_amp, adc_range, target_bw, fs): 
   
    # --- Parameters ---
    num_periods = 15
    t_stop = num_periods / f_signal
    
    # Dense time for analog appearance (oversampling_factor fs)
    t = np.arange(0, t_stop, 1/(oversampling_factor*fs))
    BW = target_bw
    
    # --- Signal (Pure Analog) ---
    signal = signal_amp * np.sin(2 * np.pi * f_signal * t)
    
    # --- Quantization Parameters ---
    v_max, v_min = adc_range, -adc_range
    num_levels = 2**bits
    lsb = (v_max - v_min) / num_levels
    
    # --- Sampled Data Logic ---
    # Pick the signal values only at sampling instances FIRST
    t_samples = t[::oversampling_factor]
    sampled_signal = signal[::oversampling_factor]
    
    # NOW QUANTIZE ONLY THE SAMPLED DATA
    indices_pre = np.floor((sampled_signal - v_min) / lsb)
    indices = np.clip(indices_pre, 0, num_levels - 1)
    sampled_data = v_min + (indices + 0.5) * lsb
    
    # For visualization (The red staircase line), we still need a quantized vector 
    # that matches the dense time 't'
    indices_dense = np.floor((signal - v_min) / lsb)
    indices_dense = np.clip(indices_dense, 0, num_levels - 1)
    quantized_dense = v_min + (indices_dense + 0.5) * lsb
    
    # CALCULATE ERROR ONLY IN SAMPLED VALUES
    sampled_error = sampled_signal - sampled_data
    
    # --- Metrics ---
    sqnr_calc = 10 * np.log10(np.var(sampled_signal) / (np.var(sampled_error)))
    sqnr_theory = 6.02 * bits + 1.76
    
    # Process Gain calculated on sampled fs
    safe_bw = max(1, min(BW, fs/2))
    process_gain = 10 * np.log10((fs/2) / safe_bw)
    sqnr_pg = sqnr_theory + process_gain
    
    # --- Print Table & Info ---
    print(f"\n{'='*45}")
    print(f"  QUANTIZATION LEVELS ({bits}-bit | {num_levels} levels)")
    print(f"  Sampled Vector Size: {len(sampled_data)} points")
    print(f"{'='*45}")
    display_limit = min(num_levels, 16)
    for i in range(display_limit):
        level_start = v_min + (i * lsb)
        v_level = v_min + (i + 0.5) * lsb
        print(f"{i:<15} | {level_start:<15.3f} | {v_level:<15.3f}")
    print(f"{'='*45}")
    
    # --- Short Text Summary ---
    print(f"  Signal Freq: {f_signal} Hz | Sampling Freq: {fs} Hz")
    print(f"{'='*45}\n")

    # --- Plotting ---
    plt.close('all')
    fig = plt.figure(figsize=(10, 8))
    
    # Subplot 1: Time Domain
    ax1 = plt.subplot(3, 1, 1)
    line_ana, = ax1.plot(t, signal, 'b--', alpha=0.3, label="Analog Input")
    line_dig = ax1.step(t, quantized_dense, where='mid', color='red', alpha=0.4, label="Digital (Held)")[0]
    line_samp = ax1.plot(t_samples, sampled_data, 'ro', markersize=4, label="Sampled Vector Points")[0]
    
    title_str = (f"LSB: {lsb:.4f} V | SQNR [TH]: {sqnr_theory:.2f} dB | "
                 f"SNQR [MEAS]: {sqnr_calc:.2f} dB\nPG: {process_gain:.2f} dB | Total: {sqnr_pg:.2f} dB")
    ax1.set_title(title_str, fontsize=10, fontweight='bold')
    ax1.legend(loc='upper right')
    ax1.set_ylabel("Magnitude (V)")
    ax1.grid(True)

    # Subplot 2: Quantization Error
    ax2 = plt.subplot(3, 1, 2, sharex=ax1)
    ax2.plot(t_samples, sampled_error, color='black', lw=0.8, label="Sampled Error")
    ax2.set_ylabel("Error (V)")
    ax2.set_ylim(-lsb*1.5, lsb*1.5)
    ax2.grid(True)

    # Subplot 3: Frequency Domain (FFT on sampled points)
    ax3 = plt.subplot(3, 1, 3)
    n_s = len(sampled_data)
    freqs = np.fft.rfftfreq(n_s, 1/fs)
    fft_mag = (np.abs(np.fft.rfft(sampled_data))/(n_s))
    
    line_fft, = ax3.plot(freqs, fft_mag, color='purple', label="Spectrum")
    ax3.axvspan(0, BW, color='green', alpha=0.1, label='Target BW')
    ax3.set_ylabel("Magnitude")
    ax3.set_xlabel("Frequency (Hz)")
    ax3.set_ylim(0, max(fft_mag)*1.2)
    ax3.grid(True)
    
    # Cursors
    cursor1 = mplcursors.cursor([line_samp], hover=False)
    cursor3 = mplcursors.cursor([line_fft], hover=False)
    
    plt.tight_layout()
    plt.show()

# --- UI Controls ---
bits_w = widgets.IntSlider(min=1, max=12, step=1, value=2, description='Bits', continuous_update=False)
amp_w = widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=2.0, description='Sig Amp', continuous_update=False)
range_w = widgets.FloatSlider(min=0.5, max=5.0, step=0.1, value=2.0, description='ADC Range', continuous_update=False)
fs_w = widgets.FloatSlider(value=f_signal*10, min=f_signal*2, max=f_signal*100, step=100, description='fs (Hz)', continuous_update=False)
bw_w = widgets.FloatSlider(value=f_signal*2, min=f_signal/5, max=f_signal*50, step=1e5, description='BW (Hz)', continuous_update=False)

out = interactive_output(plot_adc, {
    'bits': bits_w, 
    'signal_amp': amp_w, 
    'adc_range': range_w, 
    'target_bw': bw_w,
    'fs': fs_w 
})

ui = VBox([HBox([bits_w, amp_w, range_w]), HBox([fs_w, bw_w])])
display(ui, out)

Output(outputs=({'name': 'stdout', 'text': '\n=============================================\n  QUANTIZATION LE…

## 2. FFT Noise Floor and Process Gain

In [17]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive_output, VBox, HBox
from IPython.display import display, clear_output

def plot_adc_final(bits, m_power):
    # --- 0. Memory Management ---
    plt.close('all') 
    
    # --- 1. Parameters ---
    fs = 4096
    f_signal = 997  # Prime frequency to avoid bin centering artifacts
    M = 2**m_power  # FFT Size (2^n)
    
    # Time vector aligned with FFT size
    t = np.arange(M) / fs
    
    signal_amp = 2.5
    adc_range = 2.5 
    
    # --- 2. Signal & Quantization ---
    signal_sampled = signal_amp * np.sin(2 * np.pi * f_signal * t)
    
    v_max, v_min = adc_range, -adc_range
    num_levels = 2**bits
    lsb = (v_max - v_min) / num_levels
    
    # Quantization process
    indices = np.floor((signal_sampled - v_min) / lsb)
    indices = np.clip(indices, 0, num_levels - 1)
    quantized_sampled = v_min + (indices + 0.5) * lsb
    
    # Calculate error only on sampled values
    sampled_error = signal_sampled - quantized_sampled
    
    # --- 3. Metrics ---
    sqnr_measured = 10 * np.log10(np.var(signal_sampled) / (np.var(sampled_error) + 1e-15))
    sqnr_theory = 6.02 * bits + 1.76
    
    # --- 4. FFT Processing ---
    n = len(quantized_sampled)
    freqs = np.fft.rfftfreq(n, 1/fs)
    
    # Hanning window to prevent spectral leakage
    window = np.hanning(n)
    # Normalize by (sum of window / 2) to maintain correct amplitude for rfft
    fft_mag = np.abs(np.fft.rfft(quantized_sampled * window)) / (np.sum(window)/2)
    
    # Normalize peak to 0 dBc
    fft_db_raw = 20 * np.log10(fft_mag + 1e-15)
    max_val = np.max(fft_db_raw)
    fft_db_norm = fft_db_raw - max_val
    
    # Process Gain calculation (based on M samples)
    proc_gain = 10 * np.log10(M / 2)
    fft_floor_level = -sqnr_theory - proc_gain
    
    # --- 5. Plotting ---
    clear_output(wait=True)
    plt.figure(figsize=(12, 7))
    
    plt.plot(freqs, fft_db_norm, color='purple', lw=0.5, label="Quantized Spectrum", alpha=0.7)
    
    # Reference Lines
    plt.axhline(-sqnr_theory, color='red', linestyle='-', lw=2, 
                label=f"Theoretical SQNR ({sqnr_theory:.1f} dB)")
    plt.axhline(fft_floor_level, color='orange', linestyle=':', lw=2.5, 
                label=f"Expected FFT Floor (PG: -{proc_gain:.1f} dB)")
    
    plt.axhline(0, color='black', lw=1.5, label="Signal Peak (0 dBc)")
    
    # Updated Title with FFT size
    plt.title(f"ADC Discrete Analysis: FFT Size = {M}\n"
              f"Measured SQNR: {sqnr_measured:.2f} dB | Theory: {sqnr_theory:.2f} dB", 
              fontsize=13, fontweight='bold')
    
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Magnitude (dBc)")
    plt.ylim(-160, 10)
    plt.xlim(0, fs/2)
    plt.grid(True, which='both', linestyle=':', alpha=0.5)
    plt.legend(loc='lower left', framealpha=0.9)
    plt.tight_layout()
    plt.show()

# --- UI Controls ---
bits_s = widgets.IntSlider(min=8, max=14, step=1, value=12, description='ADC Bits')
m_slider = widgets.IntSlider(min=10, max=14, step=1, value=12, description='FFT Size (2^n)')

ui = VBox([HBox([bits_s, m_slider])])
out = interactive_output(plot_adc_final, {'bits': bits_s, 'm_power': m_slider})
display(ui, out)

Output()

## 3. Aliasing & Nyquist

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import mplcursors  # Added for cursor functionality
from ipywidgets import widgets, interactive_output, VBox
from scipy.fft import fft, fftfreq, fftshift

# Use ipympl for interactive backend in Jupyter/Colab
%matplotlib widget

def plot_aliasing_colored(f_signal, fs):
    # --- 0. Memory Management ---
    plt.close('all') 
    
    fn = fs / 2
    duration = 0.01
    t_analog = np.linspace(0, duration, 2000)
    t_samples = np.arange(0, duration, 1/fs)
    
    # Signal Generation
    sig_analog = 2 * np.sin(2 * np.pi * f_signal * t_analog)
    sig_samples = 2 * np.sin(2 * np.pi * f_signal * t_samples)
    
    # FFT Calculation
    N_fft = 2048
    yf = fftshift(fft(sig_samples, N_fft))
    xf = fftshift(fftfreq(N_fft, 1/fs))
    mag = 1.0/len(sig_samples) * np.abs(yf)

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(11, 8))
    plt.subplots_adjust(hspace=0.4)

    # --- Time Domain ---
    ax1.plot(t_analog, sig_analog, label="Analog (RF)", alpha=0.3, color='gray')
    # Use stem and capture the markerline for the cursor
    markerline, stemlines, baseline = ax1.stem(t_samples, sig_samples, 'r', label="Samples", basefmt=" ")
    ax1.plot(t_samples, sig_samples, 'r--', alpha=0.5)
    ax1.set_title(f"Time Domain: {f_signal}Hz Signal vs {fs}Hz Sampling")
    ax1.legend(loc='upper right')
    ax1.grid(True, alpha=0.3)

    # --- Frequency Domain ---
    line_fft, = ax2.plot(xf, mag, color='blue', lw=1.5, label="Spectrum")
    ax2.fill_between(xf, mag, color='blue', alpha=0.1)
    
    # Colored Nyquist Limits
    ax2.axvline(-fn, color='orange', linestyle='--', lw=2.5, label=f'Lower Nyquist ($-f_s/2$)')
    ax2.axvline(fn, color='green', linestyle='--', lw=2.5, label=f'Upper Nyquist ($+f_s/2$)')
    
    # Shade the "Safe" Zone 1
    ax2.axvspan(-fn, fn, color='gray', alpha=0.05)
    
    ax2.set_title("Spectrum: Observe the peak 'bouncing' off the colored limits")
    ax2.set_xlabel("Frequency [Hz]")
    ax2.set_ylabel("Magnitude")
    ax2.set_xlim(-fs, fs)
    ax2.legend(loc='upper right')
    ax2.grid(True, alpha=0.2)

    # --- Cursors Setup ---
    # Cursor 1: Interaction with sampled points
    cursor1 = mplcursors.cursor(markerline, hover=False)
    @cursor1.connect("add")
    def _(sel):
        sel.annotation.set_text(f"Time: {sel.target[0]:.4f}s\nAmp: {sel.target[1]:.2f}V")

    # Cursor 2: Interaction with frequency peaks
    cursor2 = mplcursors.cursor(line_fft, hover=False)
    @cursor2.connect("add")
    def _(sel):
        sel.annotation.set_text(f"Freq: {sel.target[0]:.1f} Hz\nMag: {sel.target[1]:.2f}")

    plt.show()

# UI Setup
f_slider = widgets.IntSlider(min=100, max=4000, step=50, value=600, description='Signal Hz')
fs_slider = widgets.IntSlider(min=500, max=5000, step=100, value=2000, description='Sample Rate')

ui = VBox([f_slider, fs_slider])
out = interactive_output(plot_aliasing_colored, {'f_signal': f_slider, 'fs': fs_slider})

display(ui, out)

Output()

## 4. SFDR values

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, VBox, interactive_output
from IPython.display import display, clear_output

def plot_adc_master_class(bits, signal_amp, adc_range, m_power):
    # --- 0. Memory Management ---
    plt.close('all') 

    # --- 1. Parameters ---
    fs = 4096
    M = 2**m_power 
    t = np.arange(M) / fs
    f_main = 997.0 
    
    # --- 2. Signal & Quantization ---
    signal = signal_amp * np.sin(2 * np.pi * f_main * t)
    
    v_max, v_min = adc_range, -adc_range
    num_levels = 2**bits
    lsb = (v_max - v_min) / num_levels
    
    # Quantization with Clipping
    indices = np.floor((signal - v_min) / lsb)
    indices = np.clip(indices, 0, num_levels - 1)
    quantized = v_min + (indices + 0.5) * lsb
    
    # --- 3. FFT & Normalization (dBFS) ---
    n = len(quantized)
    freqs = np.fft.rfftfreq(n, 1/fs)
    fft_mag = np.abs(np.fft.rfft(quantized)) / (n/2)
    fft_db_fs = 20 * np.log10(fft_mag / adc_range + 1e-15)
    
    # --- 4. Metrics ---
    carrier_idx = np.argmax(fft_db_fs)
    carrier_level = fft_db_fs[carrier_idx]
    
    # Mask carrier to find highest spur
    fft_no_carrier = fft_db_fs.copy()
    mask_width = 12
    fft_no_carrier[max(0, carrier_idx-mask_width):min(n//2, carrier_idx+mask_width)] = -200
    spur_idx = np.argmax(fft_no_carrier)
    spur_level = fft_db_fs[spur_idx]
    
    # Calculated Values for Title
    sqnr_theo_nyquist = 6.02 * bits + 1.76
    proc_gain = 10 * np.log10(M /2)
    total_expected_sqnr = sqnr_theo_nyquist + proc_gain
    fft_floor = -sqnr_theo_nyquist - proc_gain

    # --- 5. Plotting ---
    clear_output(wait=True)
    plt.figure(figsize=(12, 8))
    
    plt.plot(freqs, fft_db_fs, color='purple', lw=0.7, alpha=0.5, label="Quantized Spectrum")
    
    # Reference Lines
    plt.axhline(0, color='black', lw=2, label="0 dBFS (Full Scale)")
    plt.axhline(spur_level, color='red', linestyle='--', lw=1.5, label=f"SFDR Level ({spur_level:.1f} dBFS)")
    plt.axhline(fft_floor, color='orange', linestyle=':', lw=2, label="FFT 'Grass' Floor")

    # SFDR (dBFS) Annotation - Distance from 0 to Spur (Red)
    sfdr_dbfs = -spur_level
    plt.annotate('', xy=(fs/10, spur_level), xytext=(fs/10, 0),
                  arrowprops=dict(arrowstyle='<->', color='red', lw=1.5))
    plt.text(fs/10 + 50, spur_level/2, f"SFDR: {sfdr_dbfs:.1f} dBFS", 
              color='red', fontweight='bold', verticalalignment='center')

    # SFDR (dBc) Annotation - Distance from Carrier to Spur (Blue)
    sfdr_dbc = carrier_level - spur_level
    plt.annotate('', xy=(freqs[carrier_idx], spur_level), xytext=(freqs[carrier_idx], carrier_level),
                  arrowprops=dict(arrowstyle='<->', color='blue', lw=2))
    plt.text(freqs[carrier_idx] + 50, (carrier_level + spur_level)/2, 
              f"SFDR: {sfdr_dbc:.1f} dBc", color='blue', fontweight='bold')

    # Formatting and Title
    plt.title(f"ADC Performance: {bits}-Bit | FFT Points: {n}\n"
              f"SQNR (Nyquist): {sqnr_theo_nyquist:.1f} dB | Proc. Gain: {proc_gain:.1f} dB | "
              f"Total Expected SQNR: {total_expected_sqnr:.1f} dB", 
              fontsize=13, fontweight='bold')
    
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Magnitude (dBFS)")
    plt.ylim(min(fft_floor-20, -160), 15)
    plt.grid(True, alpha=0.2)
    plt.legend(loc='lower left', framealpha=0.9)
    plt.show()

# --- UI Controls ---
b_s = widgets.IntSlider(min=4, max=18, step=1, value=12, description='ADC Bits')
a_s = widgets.FloatSlider(min=0.1, max=10.0, step=0.1, value=4.5, description='Sig Amp (V)')
r_s = widgets.FloatSlider(min=1.0, max=10.0, step=0.5, value=5.0, description='ADC FS (V)')
m_s = widgets.IntSlider(min=12, max=16, step=1, value=12, description='FFT (2^n)')

ui = VBox([HBox([b_s, a_s]), HBox([r_s, m_s])])
out = interactive_output(plot_adc_master_class, {'bits': b_s, 'signal_amp': a_s, 'adc_range': r_s, 'm_power': m_s})

display(ui, out)

Output()